# Working with grid rotation

Rotated grids are supported in nlmod. It is implemented in the following manner:

- angrot, xorigin and yorigin (naming equal to modflow 6) are added to the attributes of the model Dataset.
- angrot is the counter-clockwise rotation angle (in degrees) of the model grid coordinate system relative to a real-world coordinate system (equal to definition in modflow 6)
- when a grid is rotated:
    - x and y (and xv and yv for a vertex grid) are in model-coordinates, instead of real-world-coordinates.
    - xc and yc are added to the Dataset and represent the cell centers in real-world coordinates (naming equal to rioxarray rotated grids)
    - the plot-methods in nlmod plot the grid in model-coordinates by default (can be overridden by the setting the parameter 'rotated' to True)
    - before intersecting with the grid, GeoDataFrames are automtically transformed to model coordinates.

When grids are not rotated, the model Dataset does not contain an attribute named 'angrot' (or its is 0). The x- and y-coordinates of the model then respresent real-world coordinates.

In this notebook we generate a model of 1 by 1 km, with a grid that is rotated 10 degrees relative to the real-world coordinates system (EPSG:28992: RD-coordinates).

In [ ]:
import os

import matplotlib

import nlmod

In [ ]:
print(f"nlmod version: {nlmod.__version__}")

nlmod.util.get_color_logger("INFO");

## Generate a model Dataset
We generate a model dataset with a rotation of 10 degrees counterclockwise.

In [ ]:
ds = nlmod.get_ds(
    [0, 1000, 0, 1000],
    angrot=10.0,
    xorigin=200_000,
    yorigin=500_000,
    delr=10.0,
    model_name="nlmod",
    model_ws="model11",
)

# We add a time dimension to ds by adding one timestamp, with a value of
# 2023-1-1. Because the first stress period is steady state (default value of
# stady_start=True) and the default stress-period length of this period is 10
# years (steady_start_perlen=3652.0) the one and only stress period of the
# simulation will start at 2013-1-1 and end at 2023-1-1. Later in this notebook,
# this period will determine the recharge-value and the ratio between summer and
# winter stage of the surface water.

ds = nlmod.time.set_ds_time(ds, time="2023-1-1", start="2013")

In [ ]:
ds.time.start

## Use a disv-grid
We call the refine method to generate a vertex grid (with the option of grid-refinement), instead of a structured grid. We can comment the next line out, to keep a structured grid, and the rest of the notebook will run without problems as well.

In [ ]:
ds = nlmod.grid.refine(ds)

## Add AHN
Download the ahn, resample to the new grid (using the method 'average') and compare.

In [ ]:
# Download AHN
extent = nlmod.grid.get_extent(ds)
ahn = nlmod.read.ahn.get_ahn3(extent)

# Resample to the grid
ds["ahn"] = nlmod.resample.structured_da_to_ds(ahn, ds, method="average")

# Compare original ahn to the resampled one
f, axes = nlmod.plot.get_map(extent, ncols=2)
norm = matplotlib.colors.Normalize()
pc = nlmod.plot.data_array(ahn, ax=axes[0], norm=norm)
nlmod.plot.colorbar_inside(pc, ax=axes[0])
pc = nlmod.plot.data_array(
    ds["ahn"], ds=ds, ax=axes[1], rotated=True, norm=norm, edgecolor="face"
)
nlmod.plot.colorbar_inside(pc, ax=axes[1]);

## Download surface water
Download BGT-polygon data, add stage information from the waterboard, and grid the polygons. Because we use a rotated grid, the bgt-polygons are in model coordinates.

In [ ]:
bgt = nlmod.read.bgt.get_bgt(extent)
bgt = nlmod.gwf.surface_water.add_stages_from_waterboards(bgt, extent=extent)
bgt = nlmod.grid.gdf_to_grid(bgt, ds).set_index("cellid")

## Download knmi-data

In [ ]:
knmi_ds = nlmod.read.knmi.get_recharge(ds)
ds.update(knmi_ds)

## Generate flopy-model
We generate a simulation and a groundwater flow model, with some standard packages.

In [ ]:
# %%
# create simulation
sim = nlmod.sim.sim(ds)

# create time discretisation
tdis = nlmod.sim.tdis(ds, sim)

# create ims
ims = nlmod.sim.ims(sim, complexity="complex")

# create groundwater flow model
gwf = nlmod.gwf.gwf(ds, sim)

# Create discretization
dis = nlmod.gwf.dis(ds, gwf)

# create node property flow
npf = nlmod.gwf.npf(ds, gwf)

# Create the initial conditions package
ic = nlmod.gwf.ic(ds, gwf, starting_head=0.0)

# Create the output control package
oc = nlmod.gwf.oc(ds, gwf)

# create recharge package
rch = nlmod.gwf.rch(ds, gwf)

## Add surface water
To the groundwater flow model

In [ ]:
# add surface water with a winter and a summer stage
# (which are both added with about half their conductance in a steady state simulation)
drn = nlmod.gwf.surface_water.gdf_to_seasonal_pkg(bgt, gwf, ds, print_input=True)

## Run the model and read the heads

In [ ]:
# run the model
nlmod.sim.write_and_run(sim, ds)

# read the heads
head = nlmod.gwf.get_heads_da(ds)

## Plot the heads in layer 1
When grid rotation is used, nlmod.plot.data_array() plots a DataArray in model coordinates. 

In [ ]:
f, ax = nlmod.plot.get_map(ds.extent)
pc = nlmod.plot.data_array(head.sel(layer=1).mean("time"), ds=ds, edgecolor="k")
cbar = nlmod.plot.colorbar_inside(pc)
bgt.plot(ax=ax, edgecolor="k", facecolor="none")

If we want to plot in realworld coordinates, we set the optional parameter 'rotated' to True.

In [ ]:
f, ax = nlmod.plot.get_map(extent)
pc = nlmod.plot.data_array(
    head.sel(layer=1).mean("time"), ds=ds, edgecolor="k", rotated=True
)
cbar = nlmod.plot.colorbar_inside(pc)
# as the surface water shapes are in model coordinates, we need to transform them
# to real-world coordinates before plotting
affine = nlmod.grid.get_affine_mod_to_world(ds)
bgt_rw = nlmod.grid.affine_transform_gdf(bgt, affine)
bgt_rw.plot(ax=ax, edgecolor="k", facecolor="none")

Export the model dataset to a netcdf-file, which you can open in qgis using 'Add mesh layer'.

In [ ]:
fname = os.path.join(ds.model_ws, "ugrid_ds.nc")
nlmod.gis.ds_to_ugrid_nc_file(ds, fname)